In [1]:
library(MLmetrics)
library(tidyverse)
library(quanteda)
library(quanteda.textmodels)
reviewdata= readRDS("reviewdata.rds")


Attaching package: ‘MLmetrics’


The following object is masked from ‘package:base’:

    Recall


── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Package version: 3.0.0
Unicode version: 13.0
ICU version: 66.1

Parallel computing: 8 of 8 threads used.

See https://quanteda.io for tutorials and examples.



In [21]:
dfm_train = reviewdata %>% 
  corpus_subset(dataset == "train") %>% 
  tokens() %>%
  dfm() %>% 
  dfm_trim(min_docfreq=0.01, docfreq_type="prop")

dfm_test = reviewdata %>% 
  corpus_subset(dataset == "test") %>% 
  tokens() %>%
  dfm() %>% 
  dfm_match(featnames(dfm_train))

myclassifier = textmodel_nb(dfm_train, 
                  docvars(dfm_train, "label"))

predicted = predict(myclassifier,newdata=dfm_test)
actual = docvars(dfm_test, "label")

results = list()
for (label in c("pos", "neg")) {
  results[[label]] = tibble(
    Precision=Precision(actual, predicted, label),
    Recall=Recall(actual, predicted, label),
    F1=F1_Score(actual, predicted, label))
}
bind_rows(results, .id="label")

label,Precision,Recall,F1
<chr>,<dbl>,<dbl>,<dbl>
pos,0.8250998,0.79368,0.8090850
neg,0.8012485,0.83176,0.8162192
